In [1]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:


sd = 20200101
ed = 20200131
sym= 'BTC'
ans_col = "movingBinary"
batch_size=64

import os,sys
import logging
import logging.config
os.environ['BASE_DIR'] =  '/Users/macico/Dropbox/btc'
os.environ['KULOKO_DIR'] = os.path.join(os.environ['BASE_DIR'], "kuloko")
os.environ['COMMON_DIR'] = os.path.join(os.environ['BASE_DIR'], "geco_commons")
os.environ['MONGO_DIR'] = os.path.join(os.environ['COMMON_DIR'] ,"mongodb")
os.environ['LOGDIR'] = os.path.join(os.environ['KULOKO_DIR'], "log")
sys.path.append(os.path.join(os.environ['KULOKO_DIR'],"items" ))

sys.path.append(os.path.join(os.path.dirname('__file__'),'..'))
os.environ['INIDIR'] = '/Users/macico/Dropbox/btc/kuloko/ini'
INIDIR=os.environ['INIDIR'] 
LOGDIR=os.environ['LOGDIR']

logging.config.fileConfig(os.path.join(INIDIR,'logconfig.ini'),defaults={'logfilename': os.path.join(LOGDIR,'logging.log')})
logger = logging.getLogger("ALEISTER")

from  gen_data import DataGen
from  feature_preprocess import featurePreprocess
from learning_executor import LearningEvaluator

# gen data 
dg =  DataGen()
dg.get_load_data_proxy()
trades = dg.get_trades(sym=sym, sd=sd, ed=ed)
Xy = dg.get_Xy(trades)

# prepro
fp = featurePreprocess(logger)
X_train, X_val, X_test, y_train, y_val, y_test = fp.convert_dataset(Xy, ans_col, test_ratio=0.4, valid_ratio=0.5)
scaler  = fp.get_scaler("minmax")
X_train,x_scaler = fp.scalingX(X_train)
X_val,_ = fp.scalingX(X_val,x_scaler)
X_test,_ = fp.scalingX(X_test,x_scaler)

train_loader, val_loader, test_loader, test_loader_one = fp.get_dataloader(X_train, y_train, X_val,y_val, X_test, y_test,batch_size)

input_dim = X_train.shape[1]


le = LearningEvaluator(name="TEST_MODEL", logger = logger)
le.get_device()

model_params = {
    'input_dim': input_dim,
    'hidden_dim' : 32,
    'layer_dim' : 64,
    'output_dim' : 1
    
}
le.get_model_instance("sdnn",model_params)

lossfn_params = {}
le.get_loss_fn("BCELogitLoss",{})

optim_params = {
    'params': le.model.parameters(),
    'weight_decay': 1e-6,
    'lr' : 1e-3}
le.get_optimizer("adam",optim_params)

le.train(train_loader, val_loader, batch_size=batch_size, n_epochs=50, n_features=1)
le.plot_losses()
predictions, values = le.evaluate(
    test_loader_one,
    batch_size=1,
    n_features=input_dim
)

/Users/macico/Dropbox/btc/aleister/src/gen_data.py:51: RuntimeWarning:

divide by zero encountered in log10

/Users/macico/Dropbox/btc/aleister/src/gen_data.py:71: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [22]:
predictions, values = le.evaluate(
    test_loader,
    batch_size=1,
    n_features=input_dim
)


In [23]:
predictions


[array([[3.2954811e-06],
        [8.1140215e-09],
        [9.6459758e-01],
        [3.0705813e-04],
        [9.5618216e-09],
        [5.1829719e-08],
        [4.0695976e-04],
        [2.4831750e-17],
        [8.1052195e-13],
        [6.0312168e-14],
        [2.4049819e-13],
        [4.6873166e-10],
        [1.4220867e-05],
        [1.4354480e-09],
        [8.5382033e-16],
        [1.7907973e-16],
        [3.6498417e-12],
        [3.1383052e-12],
        [4.6099223e-14],
        [1.3097313e-15],
        [1.1528281e-12],
        [7.7923863e-15],
        [1.4350289e-13],
        [7.0695501e-06],
        [3.8536439e-07],
        [1.4181570e-08],
        [3.2719643e-09],
        [1.1601221e-09],
        [4.3184510e-12],
        [4.8236634e-06],
        [2.2460591e-02],
        [5.6416155e-03],
        [7.9998295e-05],
        [2.3502919e-06],
        [5.0902800e-05],
        [1.0003798e-05],
        [1.4104079e-03],
        [1.0262584e-04],
        [4.0484557e-07],
        [4.5514599e-12],


/Users/macico/.pyenv/versions/py37b/lib/python3.7/site-packages/plotly/offline/offline.py:563: UserWarning:

Your filename `basic-line` didn't end with .html. Adding .html to the end of your file.

